## Deploy the Saved Model in the project to Deployment Space

In [6]:
#@hidden_cell
!rm -rf MLOps_CPD

## Imports

In [7]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_machine_learning import APIClient
from ibm_aigov_facts_client import AIGovFactsClient
#from utils import fs_utils,catalog_utils
from ibm_watson_studio_pipelines import WSPipelines
from botocore.client import Config
import ibm_boto3
import pandas as pd
import json
import os
import requests

## Pipeline Params

In [8]:
CLOUD_API_KEY = os.getenv("cloud_api_key")
MODEL_NAME = os.getenv("model_name")
DEPLOYMENT_NAME = os.getenv("deployment_name")
space_id = os.getenv("space_id")
model_id = os.getenv('model_id')
project_id = os.environ['PROJECT_ID']

In [9]:
#@hidden_cell
##MLOPS COS
ENDPOINT_URL_MLOPS = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
AUTH_ENDPOINT = "https://iam.cloud.ibm.com/oidc/token"
API_KEY_MLOPS = "W7libKUAsvOKR9sQaYBzpQiReuiVp4Vm9JWqugRsW6Rq"
CRN_MLOPS = "crn:v1:bluemix:public:cloud-object-storage:global:a/694a501fd56b4f6b9aa747128ab5267b:5e28c0cc-7d72-49b7-be0b-4e0606310e4e::"
BUCKET_MLOPS  = "mlops-cpd-onprem"
training_file_name ='german_credit_risk.csv'


space_id = "e3752c0e-4933-44d5-aa54-0199b28c8097"

cpd_url = 'https://cpd-cpd-instance.apps.ocp-675000m8qo-5pdc.cloud.techzone.ibm.com/'
cpd_apikey= 'S2iGBu4UClFmyfwcRFOhSsJR2mgNVhWgTqKan7Nl'

## Instantiate WML Client

In [10]:
WML_CREDENTIALS = {
                   "url": cpd_url,
                   "instance_id": "openshift",
                    "token" :os.environ['USER_ACCESS_TOKEN'],
                   "version" : "4.6" #If your env is CP4D 4.x.x then specify "4.x.x" instead of "4.5"
                  }

In [11]:
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.286'

In [15]:
"""__author__ == "Nijesh"
email : knijesh@sg.ibm.com

WKC and Model Inventory Utility Client


"""

import json
import os
from dataclasses import dataclass
from operator import itemgetter

import numpy as np
import pandas as pd
import requests
from ibm_watson_machine_learning import APIClient
from requests.structures import CaseInsensitiveDict


@dataclass
class CatalogUtils:
    """
    Encapsulated Catalog Utils Class to enable the use of WKC via Watson Data API.

    """

    service_url: str
    api_key: str
    project_id: str

    def get_wml_client(self):

        wml_credentials = {
            "url": self.service_url,
            "apikey": self.api_key,
        }
        wml_client = APIClient(wml_credentials)
        return wml_client

    def list_catalogs(self):
        access_token = os.environ['USER_ACCESS_TOKEN']
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        headers["Authorization"] = f"Bearer {access_token}"
        list_catalogs = requests.get(self.service_url + "/v2/catalogs", headers=headers)
        return list_catalogs.json()

    def get_catalog_id_map(self):
        result = self.list_catalogs()
        asset_map = {}
        for keys, values in result.items():
            if type(values) == list:
                for each in values:
                    asset_map[each["entity"]["name"]] = each["metadata"]["guid"]
        return asset_map

    def get_latest_asset_id(self, name):

        wml_client = self.get_wml_client()
        wml_client.set.default_project(self.project_id)
        result = wml_client.repository.get_model_details()
        result_meta = [
            each["metadata"]
            for each in result["resources"]
            if each["metadata"]["name"] == name
        ]

        my_asset_list = sorted(result_meta, key=itemgetter("created_at"), reverse=True)
        return my_asset_list[0]["id"]

    def get_revisions_asset(self, catalog_id, asset_id):
        access_token = os.environ['USER_ACESS_TOKEN']
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        headers["Authorization"] = f"Bearer {access_token}"
        search_asset = requests.get(
            self.service_url
            + f"/v2/assets/{asset_id}/revisions?catalog_id={catalog_id}",
            headers=headers,
        )
        return search_asset.json()

    def publish_asset(self, catalog_id, asset_id, name, desc, tags):
        """Publish Assets to Catalog

        Args:
            catalog_id (str): catalog id
            asset_id (str): Id of the asset to be published
            name (str): name of the asset
            desc (str): description
            tags (str): asset tag
        """
        access_token = self.create_access_token()
        url = f"{self.service_url}/v2/assets/{asset_id}/publish?project_id={self.project_id}"

        payload = json.dumps(
            {
                "catalog_id": catalog_id,
                "mode": 0,
                "metadata": {
                    "name": name,
                    "description": desc,
                    "tags": tags,
                },
            }
        )
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        print(response.text)

    def get_model_from_registry(self, name):
        """Get latest model from registry

        Args:
            name (str): name of the Model

        Returns:
            str: model id
        """
        access_token = self.create_access_token()

        url = f"{self.service_url}/v1/aigov/model_inventory/model_entries?bss_account_id=27ff418fedd6aedffb8dc6ae4164a1d2"

        payload = {}
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }
        response = requests.request("GET", url, headers=headers, data=payload)

        result = response.json()

        for each in result["results"]:
            for item in each["entity"]["modelfacts_global"]["physical_models"]:
                if (
                    item["name"] == name
                    and item["container_id"] == self.project_id
                    and item["is_deleted"] == False
                ):
                    return item["id"]

In [ ]:
ctutils = CatalogUtils(service_url=cpd_url)

## Utility Functions

In [12]:
def get_model_from_registry(catalog_name="Bank_Catalog"):
    ctutils = catalog_utils.CatalogUtils(
    service_url="https://api.dataplatform.cloud.ibm.com",
    api_key=CLOUD_API_KEY,
    project_id=project_id,
    auth_url="https://iam.cloud.ibm.com/identity/token")

    catalog_id = ctutils.get_catalog_id_map()[catalog_name]
    model_id = ctutils.get_model_from_registry(MODEL_NAME)
    print(model_id)
    
    wml_client.set.default_project(project_id)
    model_pipeline = wml_client.repository.load(model_id)
    
    return model_id,catalog_id,model_pipeline


def read_data_from_mlops_cos(key,json=False):
    def __iter__(self): return 0
    MLOPS_DATA_STORE_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id=API_KEY_MLOPS,
        ibm_service_instance_id=CRN_MLOPS,
        ibm_auth_endpoint=AUTH_ENDPOINT,
        config=Config(signature_version='oauth'),
        endpoint_url=ENDPOINT_URL_MLOPS)

    body = MLOPS_DATA_STORE_client.get_object(Bucket='mlops-asset', Key=key)['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    if json:
        gcf_df = body
    else:
        gcf_df = pd.read_csv(body)
    return gcf_df

def promote_and_deploy_model(experiment_name):
    facts_client = AIGovFactsClient(api_key=CLOUD_API_KEY, experiment_name=experiment_name, container_type="project", container_id=project_id, set_as_current_experiment=True)
    fsutils = fs_utils.FSUtils(wml_client=wml_client,catalog_id=catalog_id,project_id=project_id,bss_account_id='27ff418fedd6aedffb8dc6ae4164a1d2',space_id=space_id,
                           facts_client=facts_client)
    result = fsutils.promote_model(model_uid=model_id,model_name=MODEL_NAME)
    dep_model_uid = result['metadata']['asset_id']
    deployment_uid = fsutils.deploy_model(space_id=space_id,deployment_name=DEPLOYMENT_NAME,model_uid=dep_model_uid)
    return deployment_uid,dep_model_uid


def test_deployment(CLOUD_API_KEY,deploymet_uid):
    # NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
    API_KEY = CLOUD_API_KEY
    token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})

    mltoken = token_response.json()["access_token"]

    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

    # NOTE: manually define and pass the array(s) of values to be scored in the next line

    payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

    response_scoring = requests.post(f'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{deployment_uid}/predictions?version=2022-10-12', json=payload_scoring,
     headers={'Authorization': 'Bearer ' + mltoken})

    print(response_scoring.json())
    if response_scoring.json():
        return True

## Load the Trained Model from Model Registry

In [13]:
model_id,catalog_id,model_pipeline = get_model_from_registry("MLOps_Bank")

NameError: name 'catalog_utils' is not defined

## Promote the Model to deployment space and Deploy the Model

In [11]:
deployment_uid,dep_model_uid = promote_and_deploy_model('CreditRiskModel')

2022/11/07 11:18:24 INFO : Successfully set CreditRiskModel as current experiment
2022/11/07 11:18:24 INFO : Autolog enabled Successfully
Unsetting the project_id ...
defaultdict(<class 'list'>, {})


#######################################################################################

Synchronous deployment creation for uid: '8cd59bfb-76fa-4f79-bb10-b0d82076bd32' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2ff2d30e-9299-4865-bfbc-9fa4f977037d'
------------------------------------------------------------------------------------------------




## Model Testing on the Serving Endpoint



### Load Sample Data 

In [13]:
payload_data = read_data_from_mlops_cos('test_tfr.csv')
payload_data = payload_data.drop('Risk',axis=1)
fields = payload_data.columns.tolist()
values = [payload_data.values.tolist()[0]]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
json.dumps(payload_scoring)

'{"input_data": [{"fields": ["LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "Age", "ExistingCreditsCount", "Dependents", "OwnsProperty", "CheckingStatus", "ExistingSavings", "LoanPurpose", "Telephone", "OthersOnLoan", "Job", "EmploymentDuration", "CreditHistory", "Housing", "Sex"], "values": [[31, 1889, 3, 3, 32, 1, 1, "savings_insurance", "0_to_200", "100_to_500", "other", "none", "none", "skilled", "less_1", "credits_paid_to_date", "own", "female"]]}]}'

## Score the Endpoint

In [14]:
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.7398784195684511, 0.26012158043154887]]]}]}

### Test for Downstream Apps without using WML SDK.

In [16]:
deploy_done = test_deployment(CLOUD_API_KEY,deployment_uid)
deploy_done

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.7398784195684511, 0.26012158043154887]]]}]}


True

## Save Params in WS Pipeline

In [17]:
deployment_done = {}
deployment_done['deployment_status'] = deploy_done
deployment_done['deployment_id'] = deployment_uid
deployment_done['model_id'] = dep_model_uid
deployment_done['space_id'] = space_id

In [18]:
pipelines_client = WSPipelines.from_token(os.environ['USER_ACCESS_TOKEN'])
pipelines_client.store_results(deployment_done)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "deployment_status": .ibm_watson_studio_pipelines/results/deployment_status
    - "deployment_id": .ibm_watson_studio_pipelines/results/deployment_id
    - "model_id": .ibm_watson_studio_pipelines/results/model_id
    - "space_id": .ibm_watson_studio_pipelines/results/space_id
